<h1 style=\"text-align: center; font-size: 50px;\"> Interactive ORPO Fine-Tuning & Inference Hub for Open LLMs </h1>

This experiment provides an interactive and modular interface for selecting, downloading, fine-tuning, and evaluating large language models using ORPO (Optimal Reward Preferring Optimization).
The user can choose between state-of-the-art open LLMs like Mistral, LLaMA 2/3, and Gemma. 

# Notebook Overview
- Start Execution
- Install and Import Libraries
- Configure Settings
- Verify Assets
- Model Loader
- Inference with Default Model
- Creating the Fine-Tuned Model Name (ORPO)
- Dataset Loader
- ORPO Configuration

# Start Execution

In [1]:
import logging
import time

# Configure logger
logger: logging.Logger = logging.getLogger("run_workflow_logger")
logger.setLevel(logging.INFO)
logger.propagate = False  # Prevent duplicate logs from parent loggers

# Set formatter
formatter: logging.Formatter = logging.Formatter(
    fmt="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# Configure and attach stream handler
stream_handler: logging.StreamHandler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

In [2]:
start_time = time.time()  

logger.info("Notebook execution started.")

2025-08-06 16:20:55 - INFO - Notebook execution started.


## 📦 Install and Import Libraries

By using our Local GenAI workspace image, most of the necessary libraries to work with ORPO-based fine-tuning and evaluation already come pre-installed. In this notebook, we only need to import components for model loading, quantization, inference, and feedback visualization to run the complete ORPO workflow locally

In [3]:
%%time

%pip install -r ../requirements.txt --quiet

Note: you may need to restart the kernel to use updated packages.
CPU times: user 36 ms, sys: 12 ms, total: 48 ms
Wall time: 1.92 s


In [4]:
# ===============================
# 🧠 Core Libraries
# ===============================
import os
import sys
import yaml
from pathlib import Path
import warnings
import torch
import multiprocessing
import mlflow
from datasets import load_dataset
from typing import Dict, Any, Optional, Union, List, Tuple


# ===============================
# 🧪 Hugging Face & Transformers
# ===============================
from huggingface_hub import login
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)

# ===============================
# 🧩 Fine-tuning (ORPO + PEFT)
# ===============================
from trl import ORPOConfig, ORPOTrainer, setup_chat_format
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training

# ===============================
# 🧰 Project Modules: Core Pipeline
# ===============================
# Add the core directory to the path to import utils
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from core.selection.model_selection import ModelSelector
from core.local_inference.inference import InferenceRunner
from core.target_mapper.lora_target_mapper import LoRATargetMapper
from core.data_visualizer.feedback_visualizer import UltraFeedbackVisualizer
from core.finetuning_inference.inference_runner import AcceleratedInferenceRunner
from core.merge_model.merge_lora import merge_lora_and_save
from core.quantization.quantization_config import QuantizationSelector
from core.comparer.model_comparer import ModelComparer

# ===============================
# ⚙️ Utility Functions
# ===============================
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from src.utils import (
    load_configuration,
    load_secrets,
    load_secrets_to_env,
    configure_proxy,
    login_huggingface,
    get_project_root,
    get_configs_dir,
    get_output_dir,
    get_models_dir,
    get_fine_tuned_models_dir,
    get_model_cache_dir,
    format_model_path,
    setup_model_environment
)

2025-08-06 16:21:05.588911: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-06 16:21:05.741003: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754497265.808964    1630 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754497265.830784    1630 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754497265.977542    1630 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

[2025-08-06 16:21:11,422] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/conda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/opt/conda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


# Configure Settings

In [5]:
# Suppress Python warnings
warnings.filterwarnings("ignore")

In [6]:
CONFIG_PATH = str(get_configs_dir() / "config.yaml")
SECRETS_PATH = str(get_configs_dir() / "secrets.yaml")
MLFLOW_EXPERIMENT_NAME = "AIStudio-Fine-Tuning-Experiment"
MLFLOW_RUN_NAME = "AIStudio-Fine-Tuning-Run"
MLFLOW_MODEL_NAME = "AIStudio-Fine-Tuning-Model"
MODEL_SERVICE_RUN_NAME="AIStudio-Fine-Tuning-Service-Run"
MODEL_SERVICE_NAME="AIStudio-Fine-Tuning-Model"
MODEL_SERVICE_EXPERIMENT_NAME="AIStudio-Fine-Tuning-Experiment"

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### Configuration and Secrets Loading

In this section, we load configuration parameters and API keys from separate YAML files. This separation helps maintain security by keeping sensitive information (API keys) separate from configuration settings.

- **config.yaml**: Contains non-sensitive configuration parameters like model sources and URLs
- **secrets.yaml**: Contains sensitive API keys for services like HuggingFace
- *(Optional for Premium users)* Secrets such as API keys for services like HuggingFace can be stored as environment variables for the project and loaded into the notebook (see the project's README file for steps on how to save secrets in Secrets Manager).

In [8]:
# Load secrets from secrets.yaml file (if it exists) into environment
if Path(SECRETS_PATH).exists():
    load_secrets_to_env(SECRETS_PATH)
else:
    print(f"No secrets file found at {SECRETS_PATH}; relying on preexisting environment")

# Retrieve secrets from environment
try:
    secrets = load_secrets()
except ValueError:
    secrets = {}

# Load configuration and secrets
config = load_configuration(CONFIG_PATH)

print("✅ Configuration loaded successfully")
print("✅ Secrets loaded successfully")

No secrets file found at /home/jovyan/AI-Blueprints/generative-ai/fine-tuning-with-orpo/configs/secrets.yaml; relying on preexisting environment
✅ Configuration loaded successfully
✅ Secrets loaded successfully


### Proxy Configuration
For certain enterprise networks, a proxy configuration might be required for external service connections. If this is your case, set up the "proxy" field in your config.yaml and the following cell will configure the necessary environment variables.

In [9]:
# Configure proxy using the loaded config
configure_proxy(config)

### 🔍 Model Selector

Below are the available models for fine-tuning with ORPO.  
> ⚠️ **Note:** Make sure your Hugging Face account has access permissions for the selected model (some require manual approval).

| Model ID | Hugging Face Link |
|----------|-------------------|
| `mistralai/Mistral-7B-Instruct-v0.1` | [🔗 View on Hugging Face](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1) |
| `meta-llama/Llama-2-7b-chat-hf` | [🔗 View on Hugging Face](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) |
| `meta-llama/Meta-Llama-3-8B-Instruct` | [🔗 View on Hugging Face](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct) |
| `google/gemma-7b-it` | [🔗 View on Hugging Face](https://huggingface.co/google/gemma-7b-it) |
| `google/gemma-3-1b-it` | [🔗 View on Hugging Face](https://huggingface.co/google/gemma-3-1b-it) |


In [10]:
MODEL =  "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

### 🔐 Login to Hugging Face

To access gated models (e.g., LLaMA, Mistral, or Gemma), you must authenticate using your Hugging Face token.

Make sure your `secrets.yaml` file (or AIS Secrets Manager for Premium Users) contains the following key:

```yaml
AIS_HUGGINGFACE_API_KEY: your_huggingface_token
```

**Note**: Please refer to this project's README for detailed instuctions on how to configure secrets.

In [11]:
# Login to Hugging Face (required for downloading gated models)
try:
    login_huggingface(secrets)
    logger.info("✅ Hugging Face authentication successful")
except Exception as e:
    logger.warning(f"⚠️ Hugging Face authentication failed: {e}")
    logger.info("Some models may not be accessible if they require authentication")

2025-08-06 16:21:13 - WARNING - ⚠️ Hugging Face authentication failed: ❌ Hugging Face token not found in secrets.yaml.
2025-08-06 16:21:13 - INFO - Some models may not be accessible if they require authentication


### Attention Optimization Config
Automatically selects the most efficient attention implementation and data type (dtype) based on the GPU’s compute capability.

In [12]:
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

## Verify Assets

In [13]:
def log_asset_status(asset_path: str, asset_name: str, success_message: str, failure_message: str) -> None:
    """
    Logs the status of a given asset based on its existence.

    Parameters:
        asset_path (str): File or directory path to check.
        asset_name (str): Name of the asset for logging context.
        success_message (str): Message to log if asset exists.
        failure_message (str): Message to log if asset does not exist.
    """
    if Path(asset_path).exists():
        logger.info(f"{asset_name} is properly configured. {success_message}")
    else:
        logger.info(f"{asset_name} is not properly configured. {failure_message}")

def log_secrets_status(secrets: Dict[str, Any], success_message: str, failure_message: str) -> None:
    """
    Logs the status of secrets based on their existence.

    Parameters:
        secrets (Dict[str, Any]): Secrets retrieved to check if they exist.
        success_message (str): Message to log if secrets exists.
        failure_message (str): Message to log if secrets do not exist.
    """
    if secrets:
        logger.info(f"Project secrets are available. {success_message}")
    else:
        logger.info(f"There are no project secrets found. {failure_message}")


log_asset_status(
    asset_path=CONFIG_PATH,
    asset_name="Config",
    success_message="",
    failure_message="Please check if the configs.yaml was propely connfigured in your project on AI Studio."
)

log_secrets_status(
    secrets=secrets,
    success_message="",
    failure_message="Please check if the secrets were propely connfigured."
)

2025-08-06 16:21:14 - INFO - Config is properly configured. 
2025-08-06 16:21:14 - INFO - There are no project secrets found. Please check if the secrets were propely connfigured.


## Model Loader

In [14]:
selector = ModelSelector()
selector.select_model(MODEL)

model = selector.get_model()
tokenizer = selector.get_tokenizer()


INFO:ModelSelector:[ModelSelector] Selected model: TinyLlama/TinyLlama-1.1B-Chat-v1.0
INFO:ModelSelector:[ModelSelector] Downloading model snapshot to: /home/jovyan/AI-Blueprints/generative-ai/fine-tuning-with-orpo/models/TinyLlama__TinyLlama-1.1B-Chat-v1.0


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

INFO:ModelSelector:[ModelSelector] ✅ Model downloaded successfully to: /home/jovyan/AI-Blueprints/generative-ai/fine-tuning-with-orpo/models/TinyLlama__TinyLlama-1.1B-Chat-v1.0
INFO:ModelSelector:[ModelSelector] Loading model and tokenizer from: /home/jovyan/AI-Blueprints/generative-ai/fine-tuning-with-orpo/models/TinyLlama__TinyLlama-1.1B-Chat-v1.0
INFO:ModelSelector:[ModelSelector] Checking model for ORPO compatibility...
INFO:ModelSelector:[ModelSelector] ✅ Model 'TinyLlama/TinyLlama-1.1B-Chat-v1.0' is ORPO-compatible.


## 🤖 Inference with Default Model

The following cell runs inference using the base (non fine-tuned) model you selected earlier.

We've prepared a few prompts to test different types of reasoning and writing skills.  
You can later compare these outputs with the results generated by the fine-tuned model.

In [15]:
# 📋 Custom prompts for evaluation
prompts = [
    "I need to write some nodejs code that publishes a message to a Telegram group.",
    "What advice would you give to a frontend developer?",
    "Propose a solution that could reduce the rate of deforestation.",
    "Write a eulogy for a public figure who inspired you."
]

# ⚙️ Run inference with the selected model
runner = InferenceRunner(selector)

for idx, prompt in enumerate(prompts, 1):
    response = runner.infer(prompt)
    print(f"\n🟢 Prompt {idx}: {prompt}\n🔽 Model Response:\n{response}\n{'-'*80}")


INFO:InferenceRunner:[InferenceRunner] Detected 1 GPU, loading single-GPU config.
INFO:InferenceRunner:[InferenceRunner] Loading model and tokenizer from: /home/jovyan/AI-Blueprints/generative-ai/fine-tuning-with-orpo/models/TinyLlama__TinyLlama-1.1B-Chat-v1.0
INFO:InferenceRunner:[InferenceRunner] running: I need to write some nodejs code that publishes a message to a Telegram group....
INFO:InferenceRunner:[InferenceRunner] ✅ Inference complete.
INFO:InferenceRunner:[InferenceRunner] running: What advice would you give to a frontend developer?...
INFO:InferenceRunner:[InferenceRunner] ✅ Inference complete.
INFO:InferenceRunner:[InferenceRunner] running: Propose a solution that could reduce the rate of deforestation....
INFO:InferenceRunner:[InferenceRunner] ✅ Inference complete.
INFO:InferenceRunner:[InferenceRunner] running: Write a eulogy for a public figure who inspired you....



🟢 Prompt 1: I need to write some nodejs code that publishes a message to a Telegram group.
🔽 Model Response:
I need to write some nodejs code that publishes a message to a Telegram group. I followed the steps mentioned in the documentation to set it up, but I'm having trouble testing it. I'm using the `node-telegram-bot-api` library and I've set up a simple server that listens for incoming messages. Here's the code:

```javascript
const { Client } = require('node-telegram-bot-api');
const { token } = require('./config.json');

const bot = new
--------------------------------------------------------------------------------

🟢 Prompt 2: What advice would you give to a frontend developer?
🔽 Model Response:
What advice would you give to a frontend developer?
--------------------------------------------------------------------------------

🟢 Prompt 3: Propose a solution that could reduce the rate of deforestation.
🔽 Model Response:
Propose a solution that could reduce the rate of deforesta

INFO:InferenceRunner:[InferenceRunner] ✅ Inference complete.



🟢 Prompt 4: Write a eulogy for a public figure who inspired you.
🔽 Model Response:
Write a eulogy for a public figure who inspired you. Make sure to provide an overview of their life story and how they impacted society. Use specific examples and anecdotes to showcase their contributions and impact. Consider incorporating themes such as leadership, vision, and inspiration, and use poetic language and symbolism to convey your message. Your eulogy should conclude with a reflection on the legacy left behind by the public figure and a call to action for future generations to emulate their values and impact.
--------------------------------------------------------------------------------


## 🏷️ Creating the Fine-Tuned Model Name (ORPO)

We define a clean and consistent name for the fine-tuned version of the selected base model

In [16]:
base_model = selector.model_id
model_path = selector.format_model_path(base_model)
new_model = f"Orpo-{base_model.split('/')[-1]}-FT"
fine_tuned_name = f"Orpo-{base_model.split('/')[-1]}-FT"

fine_tuned_dir = get_fine_tuned_models_dir()
fine_tuned_dir.mkdir(parents=True, exist_ok=True)
fine_tuned_path = str(fine_tuned_dir / fine_tuned_name)

print(f"Fine-tuned model will be saved to: {fine_tuned_path}")
print(f"Directory exists: {Path(fine_tuned_path).parent.exists()}")

Fine-tuned model will be saved to: /home/jovyan/AI-Blueprints/generative-ai/fine-tuning-with-orpo/output/fine_tuned_models/Orpo-TinyLlama-1.1B-Chat-v1.0-FT
Directory exists: True


### ⚙️  Automatic Quantization Configuration

We use an intelligent selector to automatically choose the optimal quantization strategy for the hardware environment.

- `QuantizationSelector()` analyzes the number of available GPUs and their memory.
- If multiple GPUs with sufficient VRAM are detected, it applies 8-bit quantization for faster performance.
- Otherwise, it falls back to `4-bit QLoRA` using `nf4` and double quantization to reduce memory usage.

This adaptive configuration ensures efficient fine-tuning of large language models by balancing performance and hardware constraints.

In [17]:
quantization = QuantizationSelector()
bnb_config = quantization.get_config()

⚠️ Using 4-bit quantization (fallback due to lower resources).


### 🧩 PEFT Configuration (LoRA)

We define the LoRA configuration using the `LoraConfig` from PEFT (Parameter-Efficient Fine-Tuning).


In [18]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=LoRATargetMapper.get_target_modules(base_model)
)

INFO:LoRATargetMapper:✅ Matched model 'TinyLlama/TinyLlama-1.1B-Chat-v1.0' to LoRA target modules: llama


### 🧠 Load and Prepare Base Model for Training

In this step, we load the base model and tokenizer from the local path, apply the quantization configuration (`bnb_config`), prepare it for tra

In [19]:
model_vocab_size = AutoModelForCausalLM.from_pretrained(model_path).config.vocab_size
tokenizer_vocab_size = len(tokenizer)

if tokenizer_vocab_size != model_vocab_size:
    print(f"⚠️ Adjusting vocabulary ({tokenizer_vocab_size}) ≠ Model ({model_vocab_size})")
    tokenizer.pad_token = tokenizer.eos_token  
    tokenizer.save_pretrained(model_path)

In [20]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map={"": 0},
)

In [21]:
# Safely apply chat format only if tokenizer doesn't already have a chat_template
if tokenizer.chat_template is None:
    model, tokenizer = setup_chat_format(model, tokenizer)
else:
    print("⚠️ Tokenizer already has a chat_template. Skipping setup_chat_format to avoid overwriting.")


⚠️ Tokenizer already has a chat_template. Skipping setup_chat_format to avoid overwriting.


In [22]:
model = prepare_model_for_kbit_training(model)


## 📚 Dataset Loader

We use the [UltraFeedback Binarized](https://huggingface.co/datasets/HuggingFaceH4/ultrafeedback_binarized) dataset provided by Hugging Face.

This dataset contains prompts along with two model-generated responses:
- **chosen**: the response preferred by human annotators
- **rejected**: the less preferred one

For this experiment, we load a subset of the data to speed up training and evaluation.  
A fixed seed ensures reproducibility when shuffling the data.


In [23]:
dataset = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split=["train_prefs", "test_prefs"])

# 📊 Define sample sizes for a lightweight experiment
train_samples = 5000                         # Subset size for training
original_train_samples = 61135              # Total training examples in the original dataset
test_samples = int((2000 / original_train_samples) * train_samples)  # Proportional test size

# 🔀 Shuffle and sample subsets from both splits
train_subset = dataset[0].shuffle(seed=42).select(range(train_samples))
test_subset = dataset[1].shuffle(seed=42).select(range(test_samples))


### 📊 Dataset Visualization

To help understand how the dataset works, we use the `UltraFeedbackVisualizer`.

This tool logs examples from the dataset into **TensorBoard**, including:
- The **original prompt** given to the model
- The two possible answers: one **preferred by humans** and one that was **rejected**
- A simple comparison showing which response was rated better

Each example is displayed with clear labels and scores to help illustrate the kinds of outputs humans value more — **before we do any fine-tuning**.

> This is useful to explore what “good answers” look like, based on real human feedback.


In [24]:
visualizer = UltraFeedbackVisualizer(train_subset, test_subset,max_samples=20)
visualizer.run()

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:UltraFeedbackVisualizer:📊 Logging training samples (human feedback)...
INFO:UltraFeedbackVisualizer:[Example 0] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 1] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 2] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 3] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 4] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 5] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 6] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 7] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 8] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 9] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 10] ✅ Logged successfully.
INFO:UltraFeedbackVisualizer:[Example 11]

In [25]:
def process(row):
    """
    Specifies how to convert row into a tokenizable string in the expected model format
    """
    # Remove the 'messages' key to avoid conflicts with chosen/rejected format
    if 'messages' in row:
        del row['messages']
    
    # Apply chat template to chosen and rejected responses
    row["chosen"] = tokenizer.apply_chat_template(row["chosen"], tokenize=False)
    row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
    return row

dataset[0] = train_subset.map(
    process,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

dataset[1] = test_subset.map(
    process,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

print(dataset)

Map (num_proc=24):   0%|          | 0/5000 [00:00<?, ? examples/s]

Map (num_proc=24):   0%|          | 0/163 [00:00<?, ? examples/s]

[Dataset({
    features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'score_chosen', 'score_rejected'],
    num_rows: 5000
}), Dataset({
    features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'score_chosen', 'score_rejected'],
    num_rows: 163
})]


## ⚙️ ORPO Configuration

We define the training configuration using the `ORPOConfig` class from TRL (Transformers Reinforcement Learning).

This configuration controls how the model will be fine-tuned using ORPO (Offline Reinforcement Preference Optimization), a technique that aligns model outputs with human preferences.

Key parameters include:
- `learning_rate`: sets how fast the model updates (8e-6 is typical for PEFT)
- `beta`: the strength of the ORPO loss term
- `optim`: uses 8-bit optimizer for memory efficiency (paged_adamw_8bit)
- `max_steps`: controls how long training will run (e.g., 1000 steps)
- `eval_strategy` and `eval_steps`: defines how and when to evaluate during training
- `output_dir`: directory to save the trained model

> This configuration is compatible with all the selected models (e.g., Mistral, LLaMA, Gemma) and optimized for QLoRA fine-tuning on consumer or research-grade GPUs.

> **⏱️ Training time notice**  
> The duration of ORPO fine-tuning depends heavily on the model’s hyper-parameters (e.g., number of epochs, learning-rate, batch size, resolution). Reducing these values can speed up training, but expect a possible drop in quality. Tune them according to your time / quality trade-off.


In [26]:
mlflow.set_tracking_uri('/phoenix/mlflow')
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

# Ensure training output directory exists
training_output_dir = get_output_dir() / "training_results"
training_output_dir.mkdir(parents=True, exist_ok=True)

orpo_args = ORPOConfig(
    learning_rate=8e-6,
    beta=0.1,
    lr_scheduler_type="linear",
    max_length=1024,
    max_prompt_length=512,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    max_steps=2,
    eval_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=1,
    report_to=["mlflow","tensorboard"],
    output_dir=str(training_output_dir),
)

print(f"Training output directory: {training_output_dir}")
print(f"Directory exists: {training_output_dir.exists()}")

Training output directory: /home/jovyan/AI-Blueprints/generative-ai/fine-tuning-with-orpo/output/training_results
Directory exists: True


### 🚀 ORPO Trainer

We now initialize the `ORPOTrainer`, which orchestrates the fine-tuning process using the Offline Reinforcement Preference Optimization (ORPO) strategy.

It takes as input:
- The **base model**, already prepared with QLoRA and chat formatting
- The **ORPO configuration** (`orpo_args`) containing all training hyperparameters
- The **training and evaluation datasets**
- The **LoRA configuration** (`peft_config`) for parameter-efficient fine-tuning
- The **tokenizer**, passed as a `processing_class`, to apply proper formatting and padding

Once initialized, the trainer will be ready to start training with `trainer.train()`.


In [27]:
trainer = ORPOTrainer(
    model=model,
    args=orpo_args,
    train_dataset=dataset[0],
    eval_dataset=dataset[1],
    peft_config=peft_config,
    processing_class=tokenizer  
)

Map:   0%|          | 0/163 [00:00<?, ? examples/s]

Map:   0%|          | 0/163 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2784 > 2048). Running this sequence through the model will result in indexing errors
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [28]:
trainer.train()

# The training output will contain LoRA adapters in checkpoint directories
# We'll locate these adapters and merge them in the next step
print(f"Training completed. Output saved to: {orpo_args.output_dir}")
print(f"LoRA adapters will be merged and saved to: {fine_tuned_path}")

Step,Training Loss,Validation Loss,Runtime,Samples Per Second,Steps Per Second,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen,Nll Loss,Log Odds Ratio,Log Odds Chosen
1,4.925600,1.391509,81.528200,1.999000,1.006000,-0.092238,-0.095709,0.487805,0.003472,-0.957094,-0.922378,-2.804329,-2.889130,1.316913,-0.699385,0.041224
2,5.810100,1.390907,84.512500,1.929000,0.970000,-0.092148,-0.095610,0.469512,0.003462,-0.956098,-0.921475,-2.804189,-2.889081,1.316299,-0.699535,0.041081


Training completed. Output saved to: /home/jovyan/AI-Blueprints/generative-ai/fine-tuning-with-orpo/output/training_results
LoRA adapters will be merged and saved to: /home/jovyan/AI-Blueprints/generative-ai/fine-tuning-with-orpo/output/fine_tuned_models/Orpo-TinyLlama-1.1B-Chat-v1.0-FT


In [29]:
# Find the LoRA adapters in the training output directory
training_output_dir = Path(orpo_args.output_dir)

# Look for adapter_config.json in checkpoint subdirectories
adapter_configs = list(training_output_dir.rglob("adapter_config.json"))
if adapter_configs:
    # Use the directory containing adapter_config.json (typically checkpoint-X)
    lora_adapter_path = str(adapter_configs[0].parent)
    print(f"Found LoRA adapters at: {lora_adapter_path}")
    
    # Remove existing fine_tuned_path if it exists to avoid conflicts
    if Path(fine_tuned_path).exists():
        import shutil
        shutil.rmtree(fine_tuned_path)
        print(f"Removed existing directory: {fine_tuned_path}")
    
    # Merge LoRA adapters with base model and save to fine_tuned_path
    merge_lora_and_save(
        base_model_id=MODEL,
        finetuned_lora_path=lora_adapter_path
    )
    print(f"✅ Merged model saved to: {fine_tuned_path}")
else:
    print("❌ No LoRA adapters found in training output directory!")
    print("This might indicate an issue with the training process.")
    print("Available files in training output:")
    for file_path in training_output_dir.rglob("*"):
        if file_path.is_file():
            print(f"  - {file_path.relative_to(training_output_dir)}")

Found LoRA adapters at: /home/jovyan/AI-Blueprints/generative-ai/fine-tuning-with-orpo/output/training_results/checkpoint-100
Removed existing directory: /home/jovyan/AI-Blueprints/generative-ai/fine-tuning-with-orpo/output/fine_tuned_models/Orpo-TinyLlama-1.1B-Chat-v1.0-FT
🧹 Cleaning up memory...
🔄 Loading base tokenizer and model...


INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


⚠️ Tokenizer already contains a chat_template. Skipping setup.
🔗 Loading LoRA fine-tuned weights from: /home/jovyan/AI-Blueprints/generative-ai/fine-tuning-with-orpo/output/training_results/checkpoint-100
🧠 Merging LoRA weights into the base model...
💾 Saving merged model to: /home/jovyan/AI-Blueprints/generative-ai/fine-tuning-with-orpo/output/fine_tuned_models/Orpo-TinyLlama-1.1B-Chat-v1.0-FT
✅ Merge complete! Model successfully saved locally.
✅ Merged model saved to: /home/jovyan/AI-Blueprints/generative-ai/fine-tuning-with-orpo/output/fine_tuned_models/Orpo-TinyLlama-1.1B-Chat-v1.0-FT


In [30]:
# Verify and load the merged fine-tuned model for inference
final_model_path = str(get_fine_tuned_models_dir() / fine_tuned_name)

print(f"Loading fine-tuned model from: {final_model_path}")
print(f"Directory exists: {Path(final_model_path).exists()}")

# Show contents of the model directory for debugging
if Path(final_model_path).exists():
    print("\nContents of fine-tuned model directory:")
    for item in Path(final_model_path).iterdir():
        if item.is_file():
            print(f"  📄 {item.name}")
        elif item.is_dir():
            print(f"  📁 {item.name}/")
    
    # Check if it's a complete model or LoRA adapters
    has_adapter_config = (Path(final_model_path) / "adapter_config.json").exists()
    has_model_weights = any(f.suffix in ['.bin', '.safetensors'] for f in Path(final_model_path).iterdir() if f.is_file())
    has_config = (Path(final_model_path) / "config.json").exists()
    
    print(f"\nModel type analysis:")
    print(f"  • Has adapter_config.json: {has_adapter_config}")
    print(f"  • Has model weights: {has_model_weights}")
    print(f"  • Has config.json: {has_config}")
    print(f"  • Detected type: {'LoRA adapters' if has_adapter_config else 'Complete model' if has_model_weights else 'Unknown'}")

# Load the model and tokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained(final_model_path)
    model = AutoModelForCausalLM.from_pretrained(final_model_path, torch_dtype=torch.float16).cuda().eval()
    
    # Test the fine-tuned model with a sample prompt
    prompt = "Propose a solution that could reduce the rate of deforestation"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=500)

    print("\n✅ Fine-tuned Model Response:")
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    
except Exception as e:
    print(f"❌ Error loading or testing model: {e}")
    print("This indicates an issue with the model merging or saving process.")

Loading fine-tuned model from: /home/jovyan/AI-Blueprints/generative-ai/fine-tuning-with-orpo/output/fine_tuned_models/Orpo-TinyLlama-1.1B-Chat-v1.0-FT
Directory exists: True

Contents of fine-tuned model directory:
  📄 chat_template.jinja
  📄 config.json
  📄 generation_config.json
  📄 model.safetensors
  📄 special_tokens_map.json
  📄 tokenizer.json
  📄 tokenizer_config.json

Model type analysis:
  • Has adapter_config.json: False
  • Has model weights: True
  • Has config.json: True
  • Detected type: Complete model

✅ Fine-tuned Model Response:
Propose a solution that could reduce the rate of deforestation in the Amazon rainforest.


## 🔍 Model Evaluation and Comparison

After completing the ORPO fine-tuning process, we can evaluate the performance improvements by comparing responses from the base model and our fine-tuned model.

This comparison helps us understand:
- **Quality Improvements**: How the fine-tuned model generates more helpful and aligned responses
- **Training Effectiveness**: Whether the ORPO training successfully improved the model's preference alignment
- **Response Consistency**: How well the model maintains coherent and relevant outputs

The comparison uses the same test prompts to ensure fair evaluation between the base and fine-tuned models.

In [31]:
# Compare base model vs fine-tuned model using a custom comparison approach
final_model_path = str(get_fine_tuned_models_dir() / fine_tuned_name)

print("🔍 MODEL COMPARISON RESULTS")
print("=" * 80)

# Load both models for comparison
base_model_selector = ModelSelector()
base_model_selector.select_model(MODEL)
print(f"📊 Base model: {base_model_selector.model_id}")
print(f"📊 Fine-tuned model: {final_model_path}")

# Define test prompts for comparison
test_prompts = [
    "Explain the importance of sustainable agriculture.",
    "Write a Python function to check for palindromes.",
    "Describe the benefits of renewable energy sources.",
    "What are the key principles of machine learning?"
]

print("\n🚀 Running model comparison...")

# Initialize inference runners for both models
base_runner = AcceleratedInferenceRunner(
    model_selector=base_model_selector,
    dtype=torch.float16
)

ft_runner = AcceleratedInferenceRunner(
    model_selector=base_model_selector,
    finetuned_path=final_model_path,
    dtype=torch.float16
)

base_runner.load_model()
ft_runner.load_model()

# Run comparison and display results in a readable format
comparison_results = []

for idx, prompt in enumerate(test_prompts, 1):
    print(f"⚙️ Processing prompt {idx}/{len(test_prompts)}")
    
    # Get responses from both models
    base_response = base_runner.infer(prompt)
    ft_response = ft_runner.infer(prompt)
    
    # Store results
    result = {
        "prompt": prompt,
        "base_response": base_response,
        "ft_response": ft_response,
        "base_length": len(base_response),
        "ft_length": len(ft_response)
    }
    comparison_results.append(result)
    
    # Display each comparison in a structured format
    print(f"\n{'='*80}")
    print(f"📝 COMPARISON {idx}: {prompt}")
    print('='*80)
    
    print(f"\n🔸 BASE MODEL ({result['base_length']} chars):")
    print("-" * 40)
    print(base_response)
    
    print(f"\n🔹 FINE-TUNED MODEL ({result['ft_length']} chars):")
    print("-" * 40)
    print(ft_response)
    
    print(f"\n📊 METRICS:")
    print(f"   • Length difference: {result['ft_length'] - result['base_length']:+d} chars")
    if result['base_length'] > 0:
        length_change = ((result['ft_length'] - result['base_length']) / result['base_length']) * 100
        print(f"   • Length change: {length_change:+.1f}%")

print(f"\n{'='*80}")
print("📈 SUMMARY STATISTICS")
print('='*80)

# Calculate summary statistics
total_base_length = sum(r['base_length'] for r in comparison_results)
total_ft_length = sum(r['ft_length'] for r in comparison_results)
avg_base_length = total_base_length / len(comparison_results)
avg_ft_length = total_ft_length / len(comparison_results)

print(f"📊 Total prompts evaluated: {len(comparison_results)}")
print(f"📊 Average base model response length: {avg_base_length:.1f} chars")
print(f"📊 Average fine-tuned model response length: {avg_ft_length:.1f} chars")
print(f"📊 Overall length difference: {avg_ft_length - avg_base_length:+.1f} chars")

if avg_base_length > 0:
    overall_change = ((avg_ft_length - avg_base_length) / avg_base_length) * 100
    print(f"📊 Overall length change: {overall_change:+.1f}%")

print("\n✅ Model comparison completed successfully!")

INFO:ModelSelector:[ModelSelector] Selected model: TinyLlama/TinyLlama-1.1B-Chat-v1.0
INFO:ModelSelector:[ModelSelector] Downloading model snapshot to: /home/jovyan/AI-Blueprints/generative-ai/fine-tuning-with-orpo/models/TinyLlama__TinyLlama-1.1B-Chat-v1.0


🔍 MODEL COMPARISON RESULTS


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

INFO:ModelSelector:[ModelSelector] ✅ Model downloaded successfully to: /home/jovyan/AI-Blueprints/generative-ai/fine-tuning-with-orpo/models/TinyLlama__TinyLlama-1.1B-Chat-v1.0
INFO:ModelSelector:[ModelSelector] Loading model and tokenizer from: /home/jovyan/AI-Blueprints/generative-ai/fine-tuning-with-orpo/models/TinyLlama__TinyLlama-1.1B-Chat-v1.0
INFO:ModelSelector:[ModelSelector] Checking model for ORPO compatibility...
INFO:ModelSelector:[ModelSelector] ✅ Model 'TinyLlama/TinyLlama-1.1B-Chat-v1.0' is ORPO-compatible.
INFO:AcceleratedInferenceRunner:🔄 Loading tokenizer and base model from ModelSelector...


📊 Base model: TinyLlama/TinyLlama-1.1B-Chat-v1.0
📊 Fine-tuned model: /home/jovyan/AI-Blueprints/generative-ai/fine-tuning-with-orpo/output/fine_tuned_models/Orpo-TinyLlama-1.1B-Chat-v1.0-FT

🚀 Running model comparison...


INFO:AcceleratedInferenceRunner:✅ Model loaded and ready for inference.
INFO:AcceleratedInferenceRunner:🔄 Loading tokenizer and base model from ModelSelector...
INFO:AcceleratedInferenceRunner:✅ Model loaded and ready for inference.
INFO:AcceleratedInferenceRunner:🔍 Running inference for prompt (truncated): Explain the importance of sustainable agriculture....
INFO:AcceleratedInferenceRunner:✅ Inference complete.
INFO:AcceleratedInferenceRunner:🔍 Running inference for prompt (truncated): Explain the importance of sustainable agriculture....
INFO:AcceleratedInferenceRunner:✅ Inference complete.
INFO:AcceleratedInferenceRunner:🔍 Running inference for prompt (truncated): Write a Python function to check for palindromes....


⚙️ Processing prompt 1/4

📝 COMPARISON 1: Explain the importance of sustainable agriculture.

🔸 BASE MODEL (50 chars):
----------------------------------------
Explain the importance of sustainable agriculture.

🔹 FINE-TUNED MODEL (50 chars):
----------------------------------------
Explain the importance of sustainable agriculture.

📊 METRICS:
   • Length difference: +0 chars
   • Length change: +0.0%
⚙️ Processing prompt 2/4


INFO:AcceleratedInferenceRunner:✅ Inference complete.
INFO:AcceleratedInferenceRunner:🔍 Running inference for prompt (truncated): Write a Python function to check for palindromes....
INFO:AcceleratedInferenceRunner:✅ Inference complete.
INFO:AcceleratedInferenceRunner:🔍 Running inference for prompt (truncated): Describe the benefits of renewable energy sources....
INFO:AcceleratedInferenceRunner:✅ Inference complete.
INFO:AcceleratedInferenceRunner:🔍 Running inference for prompt (truncated): Describe the benefits of renewable energy sources....
INFO:AcceleratedInferenceRunner:✅ Inference complete.
INFO:AcceleratedInferenceRunner:🔍 Running inference for prompt (truncated): What are the key principles of machine learning?...
INFO:AcceleratedInferenceRunner:✅ Inference complete.
INFO:AcceleratedInferenceRunner:🔍 Running inference for prompt (truncated): What are the key principles of machine learning?...
INFO:AcceleratedInferenceRunner:✅ Inference complete.



📝 COMPARISON 2: Write a Python function to check for palindromes.

🔸 BASE MODEL (471 chars):
----------------------------------------
Write a Python function to check for palindromes. Your function should take a string as input and return True if the string is a palindrome and False otherwise. The function should ignore whitespace and punctuation when comparing the characters. Use appropriate variable names and comments to explain the logic and functionality of the function. Bonus points for adding error handling to handle cases where the input string is empty or contains non-alphabetic characters.

🔹 FINE-TUNED MODEL (420 chars):
----------------------------------------
Write a Python function to check for palindromes. Your function should take a string as input and return `True` if the input string is a palindrome; otherwise, return `False`. Your function should be well-documented, clearly explain its purpose and implementation, and include appropriate error handling for invalid inp

In [32]:
end_time: float = time.time()
elapsed_time: float = end_time - start_time
elapsed_minutes: int = int(elapsed_time // 60)
elapsed_seconds: float = elapsed_time % 60

logger.info(f"⏱️ Total execution time: {elapsed_minutes}m {elapsed_seconds:.2f}s")
logger.info("✅ Notebook execution completed successfully.")

2025-08-06 16:29:24 - INFO - ⏱️ Total execution time: 8m 28.30s
2025-08-06 16:29:24 - INFO - ✅ Notebook execution completed successfully.


Built with ❤️ using Z by HP AI Studio.